In [122]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import warnings
from datetime import *
warnings.simplefilter(action='ignore', category=FutureWarning)

In [123]:
#원하는 기간의 날짜를 리스트로 반환하는 함수
def date_range(start, end):
    '''
    startdate, enddate는 %Y%m%d 포맷으로 'YYYYMMDD' 형식
    '''
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [date.strftime("%Y.%m.%d") for date in pd.date_range(start, periods=(end-start).days+1)]
    return dates


def make_soup(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup




def last_botton_number(startdate,enddate,search_keyword,max_count_per_day):
    '''
    max_count_per_day = 크롤링할 일별 최대 뉴스의 개수
    
    
    네이버 뉴스페이지 구성

    시작페이지=1 step=10 으로 한 페이지당 10개의 뉴스를 노출시킨다.
    최대 노출 뉴스의 수는 400page 4000개의 뉴스를 노출시킨다. (그 이상은 불가능 필요하다면 검색 조건을 더 세부적으로 해야함)
    
    만약 검색한 키워드의 뉴스 총 노출수가 300개dlsep 100page를 url에 넣으면 알아서 300page로 들어가게된다

    따라서 마지막 페이지를 알아내려면 url에 page=4000을 넣으면 마지막 페이지로 이동할 수 있고
    그 soup에서 정규표현식을 통해 마지막 페이지의 숫자를 추출한다.



    날짜별로 반복:
        해당 날짜의 마지막 페이지 추출

        날짜, 마지막페이지번호, step 을 list에 추가

    return list
    '''

    url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_keyword}&sort=0&photo=0&field=0&pd=3&ds={date}&de={date}&cluster_rank=28&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{date_}to{date_},a:all&start={page}'

    #시작일 마감일 사이의 날짜들을 담은 리스트를 dates에 넣고 일 단위로 크롤링
    dates = date_range(startdate,enddate)
    
    date_lastpage_step = []

    for date in dates:  
        if max_count_per_day == -1:  
            url_ = url.format(search_keyword = search_keyword,
                            date = date,
                            date_ = date.replace('.',''),
                            page=4000)    # 네이버 뉴스페이지의 최대 노출 기사페이지 4000
            
            soup = make_soup(url_).select('a.btn') 
            
            try:    #검색 결과가 없는 경우 에러발생, 예외처리
                botton_count = int(re.findall('[0-9]+',str(soup[-1]).split(' ')[-1])[0]) * 10 - 9
                    
            except:
                print(date,'해당 일에 검색 결과가 존재하지 않습니다.')

        else:
            botton_count = max_count_per_day

        #페이지가 2이상 존재하지 않는 경우 step을 1로 설정
        step = 10 if botton_count > 10 else 1
        date_lastpage_step.append((date,botton_count,step))
    
    return date_lastpage_step



#start enddate default로 today()수정예정
def naver_news_url_crawling(search_keyword,max_count_per_day=-1,startdate='20220101',enddate='20220101'):
    '''

    max_count_per_day 하루 당 크롤링 할 최대 뉴스의 수 dafult시 (400,총 뉴스의 수) 중 작은값
    
    네이버 뉴스 웹은 page를 넘길 때 마다  (1, 11, 21, 31)순서로 바뀌고
    최대 page=4000 까지 지원한다. 페이지가 400이하일 때 page=4000을 넣으면 마지막 페이지가 나온다.
    매일 page=4000 url을 크롤링해서 마지막 페이지의 페이지버튼의 숫자를 통해 마지막 페이지를 크롤링하고
    for i in range(1, 총페이지수*10 -9, 10)으로 각 페이지마다 제목과 url 크롤링해서
    데이터프레임에 담아서 리턴 할 계획
    '''
    
    #검색하는 키워드의 띄어쓰기를 url상에 +으로 연결
    search_keyword = search_keyword.replace(' ','+')
    
    #네이버 뉴스 탭 url    
    url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_keyword}&sort=0&photo=0&field=0&pd=3&ds={date}&de={date}&cluster_rank=28&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{date_}to{date_},a:all&start={page}'

    #빈 데이터프레임을 만들고 크롤링한 데이터를 1일 단위로 concat해서 반환 할 계획
    result = []
    
    

                                #last_botton_number 파라미터는 클래스 선언하면서 추후에 수정
                                #max_count_per_day -1일 경우 최대치로 크롤링 해야하는데 수정해야함
    for date, botton_count, step in last_botton_number(startdate,enddate,search_keyword,max_count_per_day):
        for page in range(1,botton_count+1,step): 
            url_ = url.format(search_keyword = search_keyword,
                              date = date,
                              date_= date.replace('.',''),
                              page = page)

            soup = make_soup(url_).select('a.news_tit')

            #한 페이지에 최대 노출 기사수 10개 1일 단위로 크롤링을 하니
            a_page_ulr = [html['href'] for html in soup[:min(10,max_count_per_day)]]
            a_page_title = [html['title'] for html in soup[:min(10,max_count_per_day)]]
            a_page_date = [date] * len(a_page_title)

            result.append(pd.DataFrame({'date':a_page_date, 'title':a_page_title, 'url':a_page_ulr}))
    
    result = pd.concat(result,ignore_index=True).drop_duplicates()

    # url에서 신문사의 정보를 유추할 수 있는 부분을 추출
    result['news_agency'] = result['url'].apply(lambda x : x.split('/')[2]).apply(lambda x : x.replace('www.',''))
    
    return result




def main_text_crawling():
    
    
    '''
    각 신문사의 구조가 달라서 본문을 크롤링하는데에 제한이된다.
    발행수 상위 10개의 신문사의 데이터만 사용해서 본문을 크롤링하겠다.
    '''
    return None

In [124]:
df = naver_news_url_crawling(search_keyword='사회적 거리두기',max_count_per_day=5,startdate='20220401',enddate='20220501')
df

,date,title,url,news_agency
0,2022.04.01,사실상 '마지막 거리두기'…이달 중순 다시 일상회복 타진,http://yna.kr/AKR20220401030800530?did=1195m,yna.kr
1,2022.04.01,오늘 거리두기 조정안 발표…'10명 모임·자정까지 영업' 유력,http://yna.kr/AKR20220331190500530?did=1195m,yna.kr
2,2022.04.01,[속보] 10명 모임·밤12시 영업…“다음엔 거리두기 개편”,http://news.kmib.co.kr/article/view.asp?arcid=...,news.kmib.co.kr
3,2022.04.01,“2주간 확연한 감소세면 ‘실내마스크’ 제외 모든 거리두기 해제 검토”,https://news.kbs.co.kr/news/view.do?ncd=542982...,news.kbs.co.kr
4,2022.04.01,오늘 새 거리두기 발표…사적모임 10명·영업시간 밤12시 '유력',https://www.news1.kr/articles/?4634186,news1.kr
...,...,...,...,...
756,2022.05.01,"""어린이날 키캉스 어때?""…거리두기 해제로 뜨는 호텔",http://www.newsis.com/view/?id=NISX20220429_00...,newsis.com
758,2022.05.01,거리두기 해제에 고삐 풀린 밤거리…곳곳 사건사고,https://news.sbs.co.kr/news/endPage.do?news_id...,news.sbs.co.kr
764,2022.05.01,"거리두기 풀린 노동절, 서울 도심 1만명 집회 ‘대혼잡’",http://www.segye.com/content/html/2022/05/01/2...,segye.com
769,2022.05.01,거리두기 풀리자 소매치기·성범죄 기승,https://www.hankyung.com/society/article/20220...,hankyung.com


In [126]:
top10 = df['news_agency'].value_counts(normalize=True).cumsum()[:20]
#x =df.news_agency
top6 = top10.index[:6]
top6

Index(['newsis.com', 'news1.kr', 'yna.co.kr', 'yna.kr', 'edaily.co.kr',
       'ytn.co.kr'],
      dtype='object')

In [127]:
df1 = df[df['news_agency'].isin(top6)].reset_index(drop=True)
df1

,date,title,url,news_agency
0,2022.04.01,사실상 '마지막 거리두기'…이달 중순 다시 일상회복 타진,http://yna.kr/AKR20220401030800530?did=1195m,yna.kr
1,2022.04.01,오늘 거리두기 조정안 발표…'10명 모임·자정까지 영업' 유력,http://yna.kr/AKR20220331190500530?did=1195m,yna.kr
2,2022.04.01,오늘 새 거리두기 발표…사적모임 10명·영업시간 밤12시 '유력',https://www.news1.kr/articles/?4634186,news1.kr
3,2022.04.01,4일부터 2주간 '모임 10명·영업 밤 12시' 거리두기 완화,https://www.news1.kr/articles/?4634433,news1.kr
4,2022.04.01,신규확진 28만273명…'10인·밤 12시' 거리두기 완화(종합),http://www.newsis.com/view/?id=NISX20220401_00...,newsis.com
...,...,...,...,...
147,2022.04.30,마스크 쓰고 산책하는 마지막 주말…거리는 집회로 '시끌'[사회in],http://www.edaily.co.kr/news/newspath.asp?news...,edaily.co.kr
148,2022.05.01,거리두기 해제에 출근용 정장·나들이용 카메라 잘 팔려,https://www.yna.co.kr/view/AKR2022050101560000...,yna.co.kr
149,2022.05.01,"""거리두기 끝, 외출 늘면서 '신민아 핸드백' 잘 팔리네""",http://www.edaily.co.kr/news/newspath.asp?news...,edaily.co.kr
150,2022.05.01,"거리두기 완화→음주운전 증가, 전남경찰 ""단속 강화""",http://www.newsis.com/view/?id=NISX20220501_00...,newsis.com


In [ ]:
df1

In [147]:
print(df1.url[148])

https://www.yna.co.kr/view/AKR20220501015600003?input=1195m


In [148]:
#yna

url = 'https://www.yna.co.kr/view/AKR20220501015600003?input=1195m'
soup = make_soup(url)
text = soup.select('#articleWrap > div.content01.scroll-article-zone01 > div > div > article')
result = [i[4:-4]  for i in str(text).split('\n') if i[:3]=='<p>']
result

['(서울=연합뉴스) 황희경 기자 = 지난달 신종 코로나바이러스 감염증(코로나19) 방지를 위한 사회적 거리두기가 해제되면서 출근용 정장이나 나들이용 카메라 등을 찾는 사람들이 늘어나고 있다. ',
 '1일 코오롱인더스트리FnC부문에 따르면 지난달 나들이와 외출 때 가볍게 들 수 있는 크로스백이나 남성 캐주얼 재킷, 정장류 매출이 증가했다.',
 "여성 핸드백 브랜드 '쿠론'의 크로스백인 버베나 백은 4월을 기점으로 판매율이 64% 이상을 기록했다. ",
 '코오롱FnC 관계자는 "재택근무에서 출근으로 전환되면서 외출복 구입이 늘어났고, 포근해진 날씨 영향으로 나들이 및 야외 활동에 필요한 아이템이 인기를 얻고 있다"면서 "가정의 달인 5월로 접어들면서 더 많은 소비가 이뤄질 것"이라고 말했다.<br/>',
 '인기 가전 트렌드도 바뀌고 있다.',
 '지난 2년간 집에서 머무는 시간이 늘어나면서 커피머신이나 오디오 같은 실내용 가전 상품들이 주목을 받았다면 최근에는 나들이와 해외여행 등 야외활동을 준비하는 사람들이 증가하면서 카메라와 뷰티 가전 수요가 늘었다.',
 '실제 롯데백화점에서는 지난달 카메라와 뷰티 가전 매출이 전년보다 각각 2배 이상 늘어났다. ',
 '롯데백화점은 이런 변화에 맞춰 잠실점에서 미러리스 카메라와 렌즈를 3일간 무료로 체험해 불 수 있는 행사를 하고 나들이족 공략에 나섰다.',
 '또 실외 마스크 착용 의무 해제에 맞춰 피부관리기기 상품들을 가정의달 선물로 집중적으로 홍보하고 있다. ',
 '이동현 롯데백화점 H&amp;E 부문장은 "이번 가정의 달에는 전통적인 가전 상품과 함께 엔데믹(endemic·풍토병으로 굳어진 감염병) 트렌드에 맞는 소형 가전 상품들도 주목받을 것"이라고 예상했다. ',
 '이마트에서는 사회적 거리두기가 해제된 지난달 18일 이후 28일까지 건강기능식품 매출이 지난해 같은 기간보다 13.3% 늘었다. ',
 '홍삼 건강기능식품 매출은 43.8%, 유산균 등 장 건강기능식품 매출은 29.9% 각각 증가했다. '

In [ ]:
df.news_agency == 'moneys.mt.co.kr'

0    True 
1    False
2    False
3    False
4    False
     ...  
5    False
6    False
7    False
8    False
9    False
Name: news_agency, Length: 204, dtype: bool